In [1]:
%matplotlib inline
import seaborn
import numpy, scipy, matplotlib.pyplot as plt, IPython.display as ipd
import sklearn, pandas
import librosa, librosa.display
plt.rcParams['figure.figsize'] = (14, 5)

[&larr; Back to Index](index.html)

# Genre Recognition

## Load Audio

Load 30 seconds of an audio file:

In [2]:
filename_brahms = 'test.mp3'
x_brahms, sr_brahms = librosa.load(filename_brahms, duration=180)

Load 30 seconds of another audio file:

Play the audio files:

Plot the time-domain waveform of the audio signals:

In [ ]:
librosa.display.waveplot(x_brahms, sr_brahms)

Plot the spectrograms:

In [ ]:
S_brahms = librosa.feature.melspectrogram(x_brahms, sr=sr_brahms)
Sdb_brahms = librosa.amplitude_to_db(S_brahms)
librosa.display.specshow(Sdb_brahms, sr=sr_brahms, x_axis='time', y_axis='mel')
plt.colorbar()

In what ways do the time-domain waveform and spectrogram differ between the two files? What differences in musical attributes might this reflect? What additional insights are gained from plotting the spectrogram?

## Extract Features

For each segment, compute the MFCCs. Experiment with `n_mfcc` to select a different number of coefficients, e.g. 12.

We transpose the result to accommodate scikit-learn which assumes that each row is one observation, and each column is one feature dimension:

In [ ]:
mfcc_brahms.shape

Scale the features to have zero mean and unit variance:

In [ ]:
scaler = sklearn.preprocessing.StandardScaler()

In [ ]:
mfcc_brahms_scaled = scaler.fit_transform(mfcc_brahms)

Verify that the scaling worked:

In [ ]:
mfcc_brahms_scaled.mean(axis=0)

In [ ]:
mfcc_brahms_scaled.std(axis=0)

Extract MFCCs from the second audio file.

In [ ]:
mfcc_busta = librosa.feature.mfcc(x_busta, sr=sr_busta, n_mfcc=n_mfcc).T

We transpose the resulting matrix so that each row is one observation, i.e. one set of MFCCs. Note that the shape and size of the resulting MFCC matrix is equivalent to that for the first audio file:

In [ ]:
print mfcc_brahms.shape
print mfcc_busta.shape

Scale the resulting MFCC features to have approximately zero mean and unit variance. Re-use the scaler from above.

In [ ]:
mfcc_busta_scaled = scaler.transform(mfcc_busta)

Verify that the mean of the MFCCs for the second audio file is approximately equal to zero and the variance is approximately equal to one.

In [ ]:
mfcc_busta_scaled.mean(axis=0)

In [ ]:
mfcc_busta_scaled.std(axis=0)

## Train a Classifier

Concatenate all of the scaled feature vectors into one feature table.

In [ ]:
features = numpy.vstack((mfcc_brahms_scaled, mfcc_busta_scaled))

In [ ]:
features.shape

Construct a vector of ground-truth labels, where 0 refers to the first audio file, and 1 refers to the second audio file.

In [ ]:
labels = numpy.concatenate((numpy.zeros(len(mfcc_brahms_scaled)), numpy.ones(len(mfcc_busta_scaled))))

Create a classifer model object:

In [ ]:
# Support Vector Machine
model = sklearn.svm.SVC()

Train the classifier:

In [ ]:
model.fit(features, labels)

## Run the Classifier

To test the classifier, we will extract an unused 10-second segment from the earlier audio fields as test excerpts:

In [ ]:
x_brahms_test, sr_brahms = librosa.load(filename_brahms, duration=10, offset=120)

In [ ]:
x_busta_test, sr_busta = librosa.load(filename_busta, duration=10, offset=120)

Listen to both of the test audio excerpts:

In [ ]:
ipd.Audio(x_brahms_test, rate=sr_brahms)

In [ ]:
ipd.Audio(x_busta_test, rate=sr_busta)

Compute MFCCs from both of the test audio excerpts:

In [ ]:
mfcc_brahms_test = librosa.feature.mfcc(x_brahms_test, sr=sr_brahms, n_mfcc=n_mfcc).T

In [ ]:
mfcc_busta_test = librosa.feature.mfcc(x_busta_test, sr=sr_busta, n_mfcc=n_mfcc).T

In [ ]:
print mfcc_brahms_test.shape
print mfcc_busta_test.shape

Scale the MFCCs using the previous scaler:

In [ ]:
mfcc_brahms_test_scaled = scaler.transform(mfcc_brahms_test)

In [ ]:
mfcc_busta_test_scaled = scaler.transform(mfcc_busta_test)

Concatenate all test features together:

In [ ]:
features_test = numpy.vstack((mfcc_brahms_test_scaled, mfcc_busta_test_scaled))

Concatenate all test labels together:

In [ ]:
labels_test = numpy.concatenate((numpy.zeros(len(mfcc_brahms_test)), numpy.ones(len(mfcc_busta_test))))

Compute the predicted labels:

In [ ]:
predicted_labels = model.predict(features_test)

Finally, compute the accuracy score of the classifier on the test data:

In [ ]:
score = model.score(features_test, labels_test)

In [ ]:
score

Currently, the classifier returns one prediction for every MFCC vector in the test audio signal. Let's modify the procedure above such that the classifier returns a single prediction for a 10-second excerpt.

In [ ]:
predicted_labels = model.predict(mfcc_brahms_test_scaled)

In [ ]:
numpy.argmax([(predicted_labels == c).sum() for c in (0, 1)])

In [ ]:
predicted_labels = model.predict(mfcc_busta_test_scaled)

In [ ]:
numpy.argmax([(predicted_labels == c).sum() for c in (0, 1)])

## Analysis in Pandas

Read the MFCC features from the first test audio excerpt into a data frame:

In [ ]:
df_brahms = pandas.DataFrame(mfcc_brahms_test_scaled)

In [ ]:
df_brahms.shape

In [ ]:
df_brahms.head()

In [ ]:
df_busta = pandas.DataFrame(mfcc_busta_test_scaled)

Compute the pairwise correlation of every pair of 12 MFCCs against one another for both test audio excerpts. For each audio excerpt, which pair of MFCCs are the most correlated? least correlated?

In [ ]:
df_brahms.corr()

In [ ]:
df_busta.corr()

Display a scatter plot of any two of the MFCC dimensions (i.e. columns of the data frame) against one another. Try for multiple pairs of MFCC dimensions.

In [ ]:
df_brahms.plot.scatter(1, 2)

Display a scatter plot of any two of the MFCC dimensions (i.e. columns of the data frame) against one another. Try for multiple pairs of MFCC dimensions.

In [ ]:
df_busta.plot.scatter(1, 2)

Plot a histogram of all values across a single MFCC, i.e. MFCC coefficient number. Repeat for a few different MFCC numbers:

In [ ]:
df_brahms[0].plot.hist(bins=20)

In [ ]:
df_busta[0].plot.hist(bins=20)

In [ ]:
df_brahms[11].plot.hist(bins=20)

In [ ]:
df_busta[11].plot.hist(bins=20)

[&larr; Back to Index](index.html)